# Objective :- 

* A cancerous tumor is malignant, meaning it can grow and spread to other parts of the body. A benign tumor means the tumor can grow but will not spread.

* To Predict where the patient have Cancer or not 

# Step 1 :Library & Data Import

In [1]:
import os
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv(r"C:\Users\Arjun Walunj\Desktop\machine learning\KNN\bankdataset.csv")
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,999,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,999,0,nonexistent,no


#  Step 2 : Data Pre-Processing

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          32950 non-null  int64 
 1   job          32950 non-null  object
 2   marital      32950 non-null  object
 3   education    32950 non-null  object
 4   default      32950 non-null  object
 5   housing      32950 non-null  object
 6   loan         32950 non-null  object
 7   contact      32950 non-null  object
 8   month        32950 non-null  object
 9   day_of_week  32950 non-null  object
 10  duration     32950 non-null  int64 
 11  campaign     32950 non-null  int64 
 12  pdays        32950 non-null  int64 
 13  previous     32950 non-null  int64 
 14  poutcome     32950 non-null  object
 15  y            32950 non-null  object
dtypes: int64(5), object(11)
memory usage: 4.0+ MB


### Bad Rate Analysis 

In [4]:
data.y.value_counts()

no     29238
yes     3712
Name: y, dtype: int64

### Taking subset data of Number 

In [5]:
data.select_dtypes(include=[np.number]).columns.tolist()

['age', 'duration', 'campaign', 'pdays', 'previous']

In [6]:
# #Employee Numeric columns
bank_num = data[data.select_dtypes(include=[np.number]).columns.tolist()]
bank_num.head(3)

,age,duration,campaign,pdays,previous
0,49,227,4,999,0
1,37,202,2,999,1
2,78,1148,1,999,0


### Taking subset data of Category 


In [7]:
bank_dummies = data[data.select_dtypes(include=['object']).columns.tolist()]
bank_dummies.head(3)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,nonexistent,no
1,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,failure,no
2,retired,married,basic.4y,no,no,no,cellular,jul,mon,nonexistent,yes


### Converting Quality Variable to Number

In [8]:
from sklearn.preprocessing import LabelEncoder
bank_dummies=bank_dummies.apply(LabelEncoder().fit_transform)
bank_dummies.head(3)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,1,1,2,1,0,0,0,7,4,1,0
1,2,1,6,0,0,0,1,7,4,0,0
2,5,1,0,0,0,0,0,3,1,1,1


### Combine to Dataset


In [9]:
bank_combined = pd.concat([bank_num, bank_dummies],axis=1)

In [10]:
bank_combined.head()

,age,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,49,227,4,999,0,1,1,2,1,0,0,0,7,4,1,0
1,37,202,2,999,1,2,1,6,0,0,0,1,7,4,0,0
2,78,1148,1,999,0,5,1,0,0,0,0,0,3,1,1,1
3,36,120,2,999,0,0,1,6,0,2,0,1,6,1,1,0
4,59,368,2,999,0,5,0,6,0,0,0,0,4,3,1,0


# Step 3 : Data Partition

In [11]:
#Dividing data into train and test dataset
from sklearn.model_selection import train_test_split
#from random import seed

#seed(20)
train_x = bank_combined.drop(['y'],axis=1)
train_y = bank_combined['y']

# Train test split

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3,random_state=20)

In [12]:
X_train.head()

,age,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome
12016,29,14,4,999,0,1,1,2,0,2,0,0,6,0,1
9693,33,420,2,999,0,1,1,0,1,0,0,1,4,0,1
13570,39,199,4,999,0,0,0,6,0,0,0,0,1,3,1
5015,54,86,4,999,0,9,0,3,0,0,0,0,1,1,1
13147,30,924,1,999,0,10,1,3,0,2,2,0,7,3,1


#  Step 4 : Scaling Data

In [13]:
from sklearn.preprocessing import StandardScaler ,minmax_scale
scaler = StandardScaler()   # converting data into Z-scale = X-mean / Standard Deviation
#scaler.fit(X_train)         # Mean = 0 & standard Deviation =1

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
X_train[0:5]

array([[-1.06179629, -0.9454804 ,  0.50848227,  0.19476849, -0.35071478,
        -0.75596656, -0.28543646, -0.81771915, -0.51728748,  0.94380555,
        -0.45358055, -0.75527243,  0.7643457 , -1.41844178,  0.19727434],
       [-0.67667672,  0.63350965, -0.20430146,  0.19476849, -0.35071478,
        -0.75596656, -0.28543646, -1.75336219,  1.9321553 , -1.08535879,
        -0.45358055,  1.32402556, -0.09928051, -1.41844178,  0.19727434],
       [-0.09899735, -0.22598986,  0.50848227,  0.19476849, -0.35071478,
        -1.03428051, -1.92927967,  1.05356691, -0.51728748, -1.08535879,
        -0.45358055, -0.75527243, -1.39471981,  0.72506603,  0.19727434],
       [ 1.34520105, -0.66546246,  0.50848227,  0.19476849, -0.35071478,
         1.47054501, -1.92927967, -0.34989764, -0.51728748, -1.08535879,
        -0.45358055, -0.75527243, -1.39471981, -0.70393918,  0.19727434],
       [-0.9655164 ,  2.59363522, -0.56069333,  0.19476849, -0.35071478,
         1.74885895, -0.28543646, -0.34989764, 

In [15]:
X_train.mean(axis=0)

array([-1.77135085e-16,  3.38867119e-17, -1.49409593e-17,  1.05741944e-16,
        1.23224407e-17, -1.97159051e-17, -8.13281085e-17, -1.35546847e-17,
       -4.62091526e-18,  1.19835736e-16,  6.83895458e-17,  8.79514204e-17,
       -3.69673220e-18, -2.55690644e-17, -1.10901966e-16])

In [16]:
 X_train.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

# Step 5 : Model Building 

In [17]:
# train the decision tree
## Instantiate the model with 5 neighbors. 
model = KNeighborsClassifier(n_neighbors=6)
## Fit the model on the training data.
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=6)

# Step 6 : Predictions on Train Dataset

In [18]:
X_train=pd.DataFrame(X_train,columns=['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome'])
X_train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
0,-1.061796,-0.945480,0.508482,0.194768,-0.350715,-0.755967,-0.285436,-0.817719,-0.517287,0.943806,-0.453581,-0.755272,0.764346,-1.418442,0.197274
1,-0.676677,0.633510,-0.204301,0.194768,-0.350715,-0.755967,-0.285436,-1.753362,1.932155,-1.085359,-0.453581,1.324026,-0.099281,-1.418442,0.197274
2,-0.098997,-0.225990,0.508482,0.194768,-0.350715,-1.034281,-1.929280,1.053567,-0.517287,-1.085359,-0.453581,-0.755272,-1.394720,0.725066,0.197274
3,1.345201,-0.665462,0.508482,0.194768,-0.350715,1.470545,-1.929280,-0.349898,-0.517287,-1.085359,-0.453581,-0.755272,-1.394720,-0.703939,0.197274
4,-0.965516,2.593635,-0.560693,0.194768,-0.350715,1.748859,-0.285436,-0.349898,-0.517287,0.943806,2.303888,-0.755272,1.196159,0.725066,0.197274


In [19]:
y_train.reset_index(drop=True, inplace=True)

In [20]:
train=pd.concat([X_train,y_train],axis=1)
train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,-1.061796,-0.945480,0.508482,0.194768,-0.350715,-0.755967,-0.285436,-0.817719,-0.517287,0.943806,-0.453581,-0.755272,0.764346,-1.418442,0.197274,0
1,-0.676677,0.633510,-0.204301,0.194768,-0.350715,-0.755967,-0.285436,-1.753362,1.932155,-1.085359,-0.453581,1.324026,-0.099281,-1.418442,0.197274,0
2,-0.098997,-0.225990,0.508482,0.194768,-0.350715,-1.034281,-1.929280,1.053567,-0.517287,-1.085359,-0.453581,-0.755272,-1.394720,0.725066,0.197274,0
3,1.345201,-0.665462,0.508482,0.194768,-0.350715,1.470545,-1.929280,-0.349898,-0.517287,-1.085359,-0.453581,-0.755272,-1.394720,-0.703939,0.197274,0
4,-0.965516,2.593635,-0.560693,0.194768,-0.350715,1.748859,-0.285436,-0.349898,-0.517287,0.943806,2.303888,-0.755272,1.196159,0.725066,0.197274,1


In [21]:
train['Predicted']=model.predict(X_train)
train.head()

C:\Users\Arjun Walunj\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y,Predicted
0,-1.061796,-0.945480,0.508482,0.194768,-0.350715,-0.755967,-0.285436,-0.817719,-0.517287,0.943806,-0.453581,-0.755272,0.764346,-1.418442,0.197274,0,0
1,-0.676677,0.633510,-0.204301,0.194768,-0.350715,-0.755967,-0.285436,-1.753362,1.932155,-1.085359,-0.453581,1.324026,-0.099281,-1.418442,0.197274,0,0
2,-0.098997,-0.225990,0.508482,0.194768,-0.350715,-1.034281,-1.929280,1.053567,-0.517287,-1.085359,-0.453581,-0.755272,-1.394720,0.725066,0.197274,0,0
3,1.345201,-0.665462,0.508482,0.194768,-0.350715,1.470545,-1.929280,-0.349898,-0.517287,-1.085359,-0.453581,-0.755272,-1.394720,-0.703939,0.197274,0,0
4,-0.965516,2.593635,-0.560693,0.194768,-0.350715,1.748859,-0.285436,-0.349898,-0.517287,0.943806,2.303888,-0.755272,1.196159,0.725066,0.197274,1,0


# Step 7 : Model Performance Metrics

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(train['y'],train['Predicted'])
print(confusion_matrix)

[[20258   217]
 [ 1810   780]]


In [23]:
Accuracy_Train=((20258+780)/(23065)*100)
print(Accuracy_Train)

91.21179275959246


In [24]:
from sklearn.metrics import classification_report
print(classification_report(train['y'], train['Predicted']))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95     20475
           1       0.78      0.30      0.43      2590

    accuracy                           0.91     23065
   macro avg       0.85      0.65      0.69     23065
weighted avg       0.90      0.91      0.89     23065



# Step 8 : Predictions on Test Dataset

In [25]:
X_test=pd.DataFrame(X_test,columns=['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome'])
X_test.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
0,-0.484117,-0.766580,-0.560693,0.194768,1.643673,-1.034281,1.358407,1.053567,1.932155,-1.085359,-0.453581,-0.755272,0.764346,1.439569,-2.550390
1,-0.291557,-0.898811,0.152090,0.194768,-0.350715,-0.755967,-0.285436,-1.753362,-0.517287,-1.085359,-0.453581,1.324026,-0.099281,-1.418442,0.197274
2,-1.061796,1.566903,0.152090,0.194768,-0.350715,-1.034281,-1.929280,1.053567,-0.517287,0.943806,-0.453581,-0.755272,-1.826533,-1.418442,0.197274
3,0.960081,-0.716021,1.577658,0.194768,-0.350715,0.635603,-0.285436,0.585745,-0.517287,-1.085359,-0.453581,-0.755272,-0.531094,0.010563,0.197274
4,-0.195277,-0.463227,-0.560693,0.194768,-0.350715,-0.199339,-0.285436,-0.817719,1.932155,-0.070777,0.925154,1.324026,-0.099281,0.010563,0.197274


In [26]:
y_test.reset_index(drop=True, inplace=True)

In [27]:
test=pd.concat([X_test,y_test],axis=1)
test.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,-0.484117,-0.766580,-0.560693,0.194768,1.643673,-1.034281,1.358407,1.053567,1.932155,-1.085359,-0.453581,-0.755272,0.764346,1.439569,-2.550390,0
1,-0.291557,-0.898811,0.152090,0.194768,-0.350715,-0.755967,-0.285436,-1.753362,-0.517287,-1.085359,-0.453581,1.324026,-0.099281,-1.418442,0.197274,0
2,-1.061796,1.566903,0.152090,0.194768,-0.350715,-1.034281,-1.929280,1.053567,-0.517287,0.943806,-0.453581,-0.755272,-1.826533,-1.418442,0.197274,0
3,0.960081,-0.716021,1.577658,0.194768,-0.350715,0.635603,-0.285436,0.585745,-0.517287,-1.085359,-0.453581,-0.755272,-0.531094,0.010563,0.197274,0
4,-0.195277,-0.463227,-0.560693,0.194768,-0.350715,-0.199339,-0.285436,-0.817719,1.932155,-0.070777,0.925154,1.324026,-0.099281,0.010563,0.197274,0


In [28]:
test['Predicted']=model.predict(X_test)
test.head()

C:\Users\Arjun Walunj\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y,Predicted
0,-0.484117,-0.766580,-0.560693,0.194768,1.643673,-1.034281,1.358407,1.053567,1.932155,-1.085359,-0.453581,-0.755272,0.764346,1.439569,-2.550390,0,0
1,-0.291557,-0.898811,0.152090,0.194768,-0.350715,-0.755967,-0.285436,-1.753362,-0.517287,-1.085359,-0.453581,1.324026,-0.099281,-1.418442,0.197274,0,0
2,-1.061796,1.566903,0.152090,0.194768,-0.350715,-1.034281,-1.929280,1.053567,-0.517287,0.943806,-0.453581,-0.755272,-1.826533,-1.418442,0.197274,0,1
3,0.960081,-0.716021,1.577658,0.194768,-0.350715,0.635603,-0.285436,0.585745,-0.517287,-1.085359,-0.453581,-0.755272,-0.531094,0.010563,0.197274,0,0
4,-0.195277,-0.463227,-0.560693,0.194768,-0.350715,-0.199339,-0.285436,-0.817719,1.932155,-0.070777,0.925154,1.324026,-0.099281,0.010563,0.197274,0,0


# Step 9 : Model Performance Metrics on Test data 

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test['y'],test['Predicted'])
print(confusion_matrix)

[[8605  158]
 [ 858  264]]


In [30]:
Accuracy_test=((8605+264)/(9885)*100)
Accuracy_test

89.72180070814365

In [31]:
from sklearn.metrics import classification_report
print(classification_report(test['y'], test['Predicted']))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8763
           1       0.63      0.24      0.34      1122

    accuracy                           0.90      9885
   macro avg       0.77      0.61      0.64      9885
weighted avg       0.88      0.90      0.88      9885



# Cross Validation Method  - Random concept

* To identify the best value of K

In [32]:
from sklearn.metrics import accuracy_score
for K in range(25):
    K_value = K+1
    model2 = KNeighborsClassifier(n_neighbors = K_value)
    model2.fit(X_train, y_train) 
    y_pred_Train= model2.predict(X_train)
    y_pred_test = model2.predict(X_test)
    print("Train Accuracy is ", 
          round(accuracy_score(y_train,y_pred_Train)*100,0), "      ","Test Accuracy is ", round(accuracy_score(y_test,y_pred_test)*100,0),"% for K-Value:",
          K_value)

Train Accuracy is  100.0        Test Accuracy is  87.0 % for K-Value: 1
Train Accuracy is  93.0        Test Accuracy is  89.0 % for K-Value: 2
Train Accuracy is  93.0        Test Accuracy is  89.0 % for K-Value: 3
Train Accuracy is  92.0        Test Accuracy is  89.0 % for K-Value: 4
Train Accuracy is  92.0        Test Accuracy is  90.0 % for K-Value: 5
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 6
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 7
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 8
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 9
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 10
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 11
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 12
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 13
Train Accuracy is  91.0        Test Accuracy is  90.0 % for K-Value: 14


# train the decision tree
## Instantiate the model with 5 neighbors. 
model = KNeighborsClassifier(n_neighbors=8,metric='euclidean')
## Fit the model on the training data.
model.fit(X_train, y_train)

In [39]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': np.arange(1, 25)}


knn2 = KNeighborsClassifier()
knn_gscv = GridSearchCV(knn2, 
                        param_grid,
                        cv=10,
                        verbose=1)
knn_gscv.fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             verbose=1)

In [37]:
#check top performing n_neighbors value
knn_gscv.best_params_

{'n_neighbors': 11}

In [38]:
knn_gscv.best_score_

0.8997182300107784

# Finish